In [1]:
from __future__ import with_statement
from collections import defaultdict
import json
import networkx as nx
import matplotlib.pyplot as plt
import graphviz as gv
__author__    = "Uli Koehler"
__copyright__ = "Copyright 2013 Uli Koehler"
__license__   = "Apache v2.0"                                       #this was the original GO parser used in processGOTerm and parseGOOBO

In [2]:
def processGOTerm(goTerm):
    """
    In an object representing a GO term, replace single-element lists with
    their only member.
    Returns the modified object as a dictionary.
    """
    ret = dict(goTerm) #Input is a defaultdict, might express unexpected behaviour
    for key, value in ret.items():
        if len(value) == 1:
            ret[key] = value[0]
    return ret

In [3]:
def parseGOOBO(filename):
    """
    Parses a Gene Ontology dump in OBO v1.2 format.
    Yields each 
    Keyword arguments:
        filename: The filename to read
    """
    with open(filename, "r", encoding='utf-8') as infile:
        currentGOTerm = None
        for line in infile:
            line = line.strip()
            if not line: continue #Skip empty
            if line == "[Term]":
                if currentGOTerm: yield processGOTerm(currentGOTerm)
                currentGOTerm = defaultdict(list)
            elif line == "[Typedef]":
                #Skip [Typedef sections]
                currentGOTerm = None
            else: #Not [Term]
                #Only process if we're inside a [Term] environment
                if currentGOTerm is None: continue
                key, sep, val = line.partition(":")
                currentGOTerm[key].append(val.strip())
        #Add last term
        if currentGOTerm is not None:
            yield processGOTerm(currentGOTerm)

In [4]:
def getGraph(filename):       #creates a graph object from an obo file at the filename
    oboGen = parseGOOBO(filename)   #get a generator for the obo file
    G = nx.DiGraph()               #make a directed graph
    for goterm in oboGen:          #get each node in the obo file
        if 'is_a' in goterm:       #if there are is_a links...
            if type(goterm['is_a']) == list:    #if there is a list of is_a links....
                for is_a in goterm['is_a']:   #add each one as a new edge
                    G.add_edge(goterm['id'], is_a[0:10])
                    G.add_node(goterm['id'], name = goterm['name']) #add the name to the node
            else:
                G.add_edge(goterm['id'], goterm['is_a'][0:10])   #if there is only one is_a add it 
                G.add_node(goterm['id'], name = goterm['name'])
        else:
            G.add_node(goterm['id'], name = goterm['name'])       #add a node if there are no is_a links
    return G 

In [5]:
def getPathGraphs(G, source): #get a subgraph of all paths from the source to top HPO node in graph G
    bunch = []   #this will be a bunch of paths as a list
    for path in nx.all_simple_paths(G, source = source, target = 'HP:0000118'):  #118 is the top phenotype class
        bunch.append(path)     #all_simpl_paths returns a list of paths
    H = nx.DiGraph()  #create a graph to add the paths to
    for p in bunch:
        H = nx.compose(H, nx.subgraph(G,p))  #create a subgraph from each path and join in overall subgraph H
    return H

In [6]:
def noColon(hp):  #replace format HP:00000000 with HP_00000000 for graphviz
    return hp[0:2]+'_'+hp[3:11]

In [144]:
def drawSubGraph(H, color): #Converts a nx graph into a gv dot graph for display only
    dot = gv.Digraph()
    dot.graph_attr['rankdir'] = 'BT'  #arrows point upwards
    dot.graph_attr['size'] = '5'      #set the picture size
    for n in H.nodes(data = True):
        dot.node(noColon(n[0]), n[0] + " " + n[1]['name'])   #dot doesnt like colons so replace node and edge names with _
    for e in H.edges():
        dot.edge(noColon(e[0]), noColon(e[1]), color=color)
    return dot

In [95]:
def getPic(searchTerm, G, color):  #gets the HPO subgraph for a term in an old and new ontology version
    H = getPathGraphs(G, searchTerm)
    return drawSubGraph(H, color)   #return the dot

In [86]:
directory = 'C:/Users/Andrew Devereau/Documents/Python Scripts/'
newFile = directory + 'hp051016.obo'
oldFile = directory + 'hpoDMC.obo'
newG = getGraph(newFile)
oldG = getGraph(oldFile)
print (newG.number_of_nodes(), newG.number_of_edges())
print (oldG.number_of_nodes(), oldG.number_of_edges())

11939 15595
11067 14698


The code below gets old and new versions of subgraph for each code and merges them to show changes. Note though that this does not show when a parent node has the same HPO code but its name has changed as the old name is not displayed, thus some results may show the same subgraph for old and new

In [213]:
with open('parentChanges.txt', 'r') as infile:
    hpoChangeList = [x.strip() for x in infile]   #gp through the list of changed HPO terms
    for searchTerm in hpoChangeList:
        dot1 = getPic(searchTerm, oldG, 'blue')   #for each get the old(blue) and new (red) subGraph
        dot2 = getPic(searchTerm, newG, 'red')
        dot1.subgraph(dot2)                        #combine them and save as a pdf
        dot1.render('C:/Users/Andrew Devereau/pdf/' + noColon(searchTerm))